In [ ]:
import boto3
from boto3.resources.collection import ResourceCollection
from datetime import datetime
import pytz

In [3]:
session = boto3.Session(
    aws_access_key_id="",
    aws_secret_access_key="",
    aws_session_token=""
)

s3_client = session.client("s3")

bucket_trusted = session.resource("s3").Bucket("bucket-biosentinel-trusted")
bucket_trusted_name = "bucket-biosentinel-trusted"

bucket_client = session.resource("s3").Bucket("bucket-biosentinel-client")
bucket_client_name = "bucket-biosentinel-client"

In [4]:
def filter_objects(bucket_name: str, sensor_to_move: str):
    print(f"Filtering objects for sensor: {sensor_to_move}")
    objects_found = []
    continuation_token = None

    while True:
        if continuation_token:
            response = s3_client.list_objects_v2(
                Bucket=bucket_name,
                ContinuationToken=continuation_token
            )
        else:
            response = s3_client.list_objects_v2(Bucket=bucket_name)

        if "Contents" in response:
            for obj in response["Contents"]:
                if sensor_to_move in obj["Key"]:
                    objects_found.append(obj["Key"])

        if response.get("IsTruncated"):
            continuation_token = response.get("NextContinuationToken")
        else:
            break

    if not objects_found:
        print(f"No objects were found with {sensor_to_move}")
    
    print(f"Objects found for {sensor_to_move}: {objects_found}")
    return objects_found

# Função para verificar se a pasta existe
def exists_folder(bucket: ResourceCollection, path: str):
    objects_in_path = list(bucket.objects.filter(Prefix=path))
    return True if objects_in_path else False

# Função para copiar objetos para o bucket do cliente para todos os sensores
def copy_all_sensors_to_client(date_to_move: str):
    try:
        date_to_move_obj = datetime.strptime(date_to_move, '%Y-%m-%d')
        date_to_move_str = date_to_move_obj.strftime('%Y-%m-%d')
    except ValueError:
        raise ValueError("Date must be in the format YYYY-MM-DD")

    # Dicionário de sensores com seus respectivos prefixos
    sensors = {
        "geolocation": "geolocation/",
        "heart_rate-electric_pulse": "heart_rate-electric_pulse/",
        "umid-temp": "umid-temp/",
        "presence": "presence/",
        "presenca": "presence/",
        "sound": "sound/",
        "som": "sound/",
        "temp-pressure": "temp-pressure/"
    }

    for sensor_name, destiny_path in sensors.items():
        print(f"\nProcessing sensor: {sensor_name}")
        objects_to_copy = filter_objects(bucket_trusted_name, sensor_name)

        for object_key in objects_to_copy:
            # Verifica se a data no nome do arquivo corresponde à data alvo
            if date_to_move_str in object_key:
                specific_destiny_path = f"{destiny_path}{date_to_move_str}/"

                # Verifica se a pasta existe, senão cria
                if not exists_folder(bucket_client, specific_destiny_path):
                    s3_client.put_object(
                        Bucket=bucket_client_name,
                        Key=f"{specific_destiny_path}"
                    )
                    print(f"{specific_destiny_path} created in bucket_client")

                # Verifica se o arquivo já existe no destino para evitar duplicação
                existing_files = s3_client.list_objects_v2(
                    Bucket=bucket_client_name,
                    Prefix=f"{specific_destiny_path}{object_key.split('/')[-1]}"
                )
                if 'Contents' in existing_files:
                    print(f"{object_key} already exists in bucket_client, skipping copy.")
                    continue

                # Copia o objeto para o destino
                source = {"Bucket": bucket_trusted_name, "Key": object_key}
                destination = {"Bucket": bucket_client_name, "Key": f"{specific_destiny_path}{object_key.split('/')[-1]}"}
                s3_client.copy_object(CopySource=source, Bucket=destination["Bucket"], Key=destination["Key"])
                print(f"{object_key} copied to {destination['Key']} in bucket_client")

# Exemplo de chamada para copiar objetos de todos os sensores para a data especificada
date_to_move = "2024-11-06"
copy_all_sensors_to_client(date_to_move)

Filtering objects
['presence/', 'presence/presence_2024-09-10_00-00-00_1.json', 'presence/presence_2024-09-10_00-00-10_1.json', 'presence/presence_2024-09-10_00-00-20_1.json', 'presence/presence_2024-09-10_00-00-30_1.json', 'presence/presence_2024-09-10_00-00-40_1.json', 'presence/presence_2024-09-10_00-00-50_1.json', 'presence/presence_2024-09-10_00-01-00_1.json', 'presence/presence_2024-09-10_00-01-10_1.json', 'presence/presence_2024-09-10_00-01-20_1.json', 'presence/presence_2024-09-10_00-01-30_1.json', 'presence/presence_2024-09-10_00-01-40_1.json', 'presence/presence_2024-09-10_00-01-50_1.json', 'presence/presence_2024-09-10_00-02-00_1.json', 'presence/presence_2024-09-10_00-02-10_1.json', 'presence/presence_2024-09-10_00-02-20_1.json', 'presence/presence_2024-09-10_00-02-30_1.json', 'presence/presence_2024-09-10_00-02-40_1.json', 'presence/presence_2024-09-10_00-02-50_1.json', 'presence/presence_2024-09-10_00-03-00_1.json', 'presence/presence_2024-09-10_00-03-10_1.json', 'presenc